In [1]:
import http.client
import json
import pandas as pd
import requests
import os
import pickle
import time

C:\Users\Arda\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [ ]:
class api_management(http.client.HTTPSConnection):
    
    def __init__(self):
        
        http.client.HTTPSConnection.__init__(self,"v3.football.api-sports.io")


        self.headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': "*****************"
        }

    
    def quota_info(self):
        
        self.request("GET", "/status", headers=self.headers)
        res = self.getresponse()
        data = res.read()
        status=json.loads(data.decode("utf-8"))
        self.used=status["response"]["requests"]["current"]
        self.quota=status["response"]["requests"]["limit_day"]
        self.remains= self.quota-self.used
        print("used : {}, quota : {}, remains : {}".format(self.used,self.quota,self.remains))
        return self.remains

    
    def get_leagues(self):
    
        self.request("GET", "/leagues", headers=self.headers)
        res = self.getresponse()
        data = res.read()
        league_res=json.loads(data.decode("utf-8"))["response"]
        league_df=pd.json_normalize(league_res,record_path="seasons",meta=[["league","id"],["league","name"], ["country","name"],["league","type"] ])
        only_leagues= league_df[league_df["league.type"]=="League"][
        ["year","coverage.fixtures.events","coverage.fixtures.statistics_fixtures",
         "league.name","league.id","country.name"]]
        only_leagues["Coverage"]=  only_leagues[[
        "coverage.fixtures.events",
        "coverage.fixtures.statistics_fixtures"]].applymap(lambda x : 1 if x==True else 0).sum(axis=1)
        leagues_acc= only_leagues[only_leagues["Coverage"]==2]
        self.quota_info()
        return leagues_acc
    
    def get_fixtures(self,season,league):
   
        self.request("GET", "/fixtures?season={}&league={}".format(season,league), headers=self.headers)

        res = self.getresponse()
        data = res.read()
        fixture_res=json.loads(data.decode("utf-8"))["response"]
        self.quota_info()
        return fixture_res
    
    def  fix_stat(self,fixture):
        self.request("GET", "/fixtures/statistics?fixture={}".format(fixture), headers=self.headers)

        res = self.getresponse()
        data = res.read()
        stat_res=json.loads(data.decode("utf-8"))["response"]
        for k in range(len(stat_res)):
            stat_res[k]["statistics"]={i["type"]:i["value"] for i in stat_res[k]["statistics"]}
            stat_res[k]["fixture"]= {"id":fixture}
        
        return stat_res
    def fix_events(self,fixture):
        self.request("GET", "/fixtures/events?fixture={}".format(fixture), headers=self.headers)

        res = self.getresponse()
        data = res.read()
        event_res=json.loads(data.decode("utf-8"))["response"]
        for k in range(len(event_res)):
            event_res[k]["fixture"]= {"id":fixture}
        return event_res

## Data Download Flow

In [3]:
selected_leagues=[61, 144,  71,  39,  78, 135,  88,  94, 140,  62, 203, 197,  79,
       188, 218, 119,  40,  41,  42,  98,  72, 141, 136, 103,  89, 113,
       169, 207, 210, 235, 292, 307, 106, 265, 323, 475]

In [4]:
api= api_management()


In [5]:
api

In [24]:
data_dir="C:/Users/Arda/Downloads/Upwork/Portfolio/Betting_Asistant/data/"

In [25]:
#Leagues with sufficient features

leagues=api.get_leagues()
leagues.to_csv("{}leagues.csv".format(data_dir),index=False)

used : 6, quota : 7500, remains : 7494


C:\Users\Arda\AppData\Local\Temp\ipykernel_4220\981199331.py:39: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  "coverage.fixtures.statistics_fixtures"]].applymap(lambda x : 1 if x==True else 0).sum(axis=1)


In [26]:
#Because we download all data that we need, we can import league dataframe from our local pc.
leagues

,year,coverage.fixtures.events,coverage.fixtures.statistics_fixtures,league.name,league.id,country.name,Coverage
13,2015,True,True,Ligue 1,61,France,2
14,2016,True,True,Ligue 1,61,France,2
15,2017,True,True,Ligue 1,61,France,2
16,2018,True,True,Ligue 1,61,France,2
17,2019,True,True,Ligue 1,61,France,2
...,...,...,...,...,...,...,...
5980,2023,True,True,Canadian Premier League,479,Canada,2
5981,2024,True,True,Canadian Premier League,479,Canada,2
6036,2024,True,True,Damallsvenskan,549,Sweden,2
6970,2023,True,True,Copa de la Liga Profesional,1032,Argentina,2


### Fixtures

In [27]:
#We will get fixtures for given league and season
#But we need to stay in allowed api call rate 450/60

In [28]:
#We will use rate of change in time/calls. Becouse we will observe given time change according to one api call
call_rate= 60/300
call_rate

0.2

In [29]:
os.listdir(data_dir)

['datamart.csv',
 'fixture_event_df.csv',
 'fixture_stat.csv',
 'leagues.csv',
 'saved_fixtures.csv']

In [30]:

try:
    
    saved_fixtures_df=pd.read_csv("{}saved_fixtures.csv".format(data_dir))
except:
    saved_fixtures_df=[]
    


C:\Users\Arda\AppData\Local\Temp\ipykernel_4220\3306976125.py:3: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  saved_fixtures_df=pd.read_csv("{}saved_fixtures.csv".format(data_dir))


In [31]:
saved_fixtures_df

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings
0,185545,"Fredy Fautrel, France",UTC,2015-08-07T18:30:00+00:00,1438972200,1.438972e+09,1.438976e+09,NaN,Stade Pierre Mauroy,Lille,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,185546,"Wilfried Bien, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Stade de la Beaujoire,Nantes,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,185547,"Hakim Ben el Salem Hadj, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,660.0,Stade de la Mosson,Montpellier,...,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,185548,"Lionel Jaffredo, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Allianz Rivera,Nice,...,1.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,185549,"Frank Schneider, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,667.0,Orange Velodrome,Marseille,...,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103985,1354223,Edina Alves Batista,UTC,2025-03-03T23:00:00+00:00,1741042800,1.741043e+09,1.741046e+09,20813.0,MorumBIS,"São Paulo, São Paulo",...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,7.0,True
103986,1357839,Matheus Delgado Candançan,UTC,2025-03-09T21:30:00+00:00,1741555800,1.741556e+09,1.741559e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,9.0,True
103987,1358545,Flavio Rodrigues De Souza,UTC,2025-03-11T00:35:00+00:00,1741653300,1.741653e+09,1.741657e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,9.0,True
103988,1360313,NaN,UTC,2025-03-16T21:30:00+00:00,1742160600,NaN,NaN,258.0,Allianz Parque,"São Paulo, São Paulo",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [32]:
saved_fixtures_df=saved_fixtures_df[saved_fixtures_df["league.id"].isin(selected_leagues)]

In [33]:
saved_fixtures_df.columns

Index(['fixture.id', 'fixture.referee', 'fixture.timezone', 'fixture.date',
       'fixture.timestamp', 'fixture.periods.first', 'fixture.periods.second',
       'fixture.venue.id', 'fixture.venue.name', 'fixture.venue.city',
       'fixture.status.long', 'fixture.status.short', 'fixture.status.elapsed',
       'league.id', 'league.name', 'league.country', 'league.logo',
       'league.season', 'league.round', 'teams.home.id', 'teams.home.name',
       'teams.home.winner', 'teams.away.id', 'teams.away.name',
       'teams.away.winner', 'goals.home', 'goals.away', 'score.halftime.home',
       'score.halftime.away', 'score.fulltime.home', 'score.fulltime.away',
       'score.extratime.home', 'score.extratime.away', 'score.penalty.home',
       'score.penalty.away', 'fixture.status.extra', 'league.standings'],
      dtype='object')

In [34]:
#MAX season for each leagues
max_fixtures=saved_fixtures_df.groupby("league.id",as_index=False)["league.season"].max().rename({"league.season":"max_season"},axis=1)
max_fixtures

,league.id,max_season
0,39,2024
1,40,2024
2,41,2024
3,42,2024
4,61,2024
5,62,2024
6,71,2024
7,72,2024
8,78,2024
9,79,2024


In [35]:
last_seasons=leagues.merge(max_fixtures)
last_seasons_v1=last_seasons[last_seasons.max_season<=last_seasons["year"]].sort_values(["league.id","year"]).drop("max_season",axis=1)
last_seasons_v1

,year,coverage.fixtures.events,coverage.fixtures.statistics_fixtures,league.name,league.id,country.name,Coverage
40,2024,True,True,Premier League,39,England,2
174,2024,True,True,Championship,40,England,2
184,2024,True,True,League One,41,England,2
192,2024,True,True,League Two,42,England,2
9,2024,True,True,Ligue 1,61,France,2
102,2024,True,True,Ligue 2,62,France,2
29,2024,True,True,Serie A,71,Brazil,2
200,2024,True,True,Serie B,72,Brazil,2
50,2024,True,True,Bundesliga,78,Germany,2
132,2024,True,True,2. Bundesliga,79,Germany,2


In [36]:
prior_saved_fixtures=saved_fixtures_df.merge(max_fixtures)
prior_saved_fixtures=prior_saved_fixtures[prior_saved_fixtures["league.season"]<prior_saved_fixtures.max_season]
prior_saved_fixtures

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings,max_season
0,185545,"Fredy Fautrel, France",UTC,2015-08-07T18:30:00+00:00,1438972200,1.438972e+09,1.438976e+09,NaN,Stade Pierre Mauroy,Lille,...,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
1,185546,"Wilfried Bien, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Stade de la Beaujoire,Nantes,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
2,185547,"Hakim Ben el Salem Hadj, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,660.0,Stade de la Mosson,Montpellier,...,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
3,185548,"Lionel Jaffredo, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Allianz Rivera,Nice,...,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
4,185549,"Frank Schneider, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,667.0,Orange Velodrome,Marseille,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93054,1181854,Raphael Claus,UTC,2024-03-17T23:15:00+00:00,1710717300,1.710717e+09,1.710721e+09,10494.0,Estádio Urbano Caldeira,"Santos, São Paulo",...,0.0,0.0,0.0,0.0,0.0,4.0,2.0,NaN,True,2025
93055,1186176,Matheus Delgado Candançan,UTC,2024-03-27T23:30:00+00:00,1711582200,1.711582e+09,1.711586e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,0.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,True,2025
93056,1186175,Edina Alves Batista,UTC,2024-03-29T00:35:00+00:00,1711672500,1.711672e+09,1.711676e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,True,2025
93057,1190416,Flavio Rodrigues De Souza,UTC,2024-03-31T21:00:00+00:00,1711918800,1.711919e+09,1.711922e+09,10494.0,Estádio Urbano Caldeira,"Santos, São Paulo",...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,True,2025


In [37]:
saved_fixtures_download=[]
quota=api.quota_info()-10

for i in range(last_seasons_v1.shape[0]):
    season=last_seasons_v1.iloc[i]["year"]
    league=last_seasons_v1.iloc[i]["league.id"]
    current=time.time()
    saved_fixtures_download=saved_fixtures_download+api.get_fixtures(season,league)
    diff=time.time()-current
    if diff < call_rate:
        time.sleep(0.2)
    print(quota,league,season,i)




used : 6, quota : 7500, remains : 7494
used : 6, quota : 7500, remains : 7494
7484 39 2024 0
used : 6, quota : 7500, remains : 7494
7484 40 2024 1
used : 6, quota : 7500, remains : 7494
7484 41 2024 2
used : 6, quota : 7500, remains : 7494
7484 42 2024 3
used : 6, quota : 7500, remains : 7494
7484 61 2024 4
used : 6, quota : 7500, remains : 7494
7484 62 2024 5
used : 6, quota : 7500, remains : 7494
7484 71 2024 6
used : 6, quota : 7500, remains : 7494
7484 72 2024 7
used : 6, quota : 7500, remains : 7494
7484 78 2024 8
used : 6, quota : 7500, remains : 7494
7484 79 2024 9
used : 6, quota : 7500, remains : 7494
7484 88 2024 10
used : 6, quota : 7500, remains : 7494
7484 89 2024 11
used : 6, quota : 7500, remains : 7494
7484 94 2024 12
used : 6, quota : 7500, remains : 7494
7484 98 2025 13
used : 6, quota : 7500, remains : 7494
7484 103 2024 14
used : 6, quota : 7500, remains : 7494
7484 106 2024 15
used : 6, quota : 7500, remains : 7494
7484 113 2024 16
used : 6, quota : 7500, remains :

In [38]:
post_saved_fixtures=pd.json_normalize(saved_fixtures_download).drop(['teams.home.logo','teams.away.logo','league.flag'],axis=1)


In [39]:
prior_saved_fixtures

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings,max_season
0,185545,"Fredy Fautrel, France",UTC,2015-08-07T18:30:00+00:00,1438972200,1.438972e+09,1.438976e+09,NaN,Stade Pierre Mauroy,Lille,...,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
1,185546,"Wilfried Bien, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Stade de la Beaujoire,Nantes,...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
2,185547,"Hakim Ben el Salem Hadj, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,660.0,Stade de la Mosson,Montpellier,...,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
3,185548,"Lionel Jaffredo, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Allianz Rivera,Nice,...,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
4,185549,"Frank Schneider, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,667.0,Orange Velodrome,Marseille,...,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93054,1181854,Raphael Claus,UTC,2024-03-17T23:15:00+00:00,1710717300,1.710717e+09,1.710721e+09,10494.0,Estádio Urbano Caldeira,"Santos, São Paulo",...,0.0,0.0,0.0,0.0,0.0,4.0,2.0,NaN,True,2025
93055,1186176,Matheus Delgado Candançan,UTC,2024-03-27T23:30:00+00:00,1711582200,1.711582e+09,1.711586e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,0.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,True,2025
93056,1186175,Edina Alves Batista,UTC,2024-03-29T00:35:00+00:00,1711672500,1.711672e+09,1.711676e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,True,2025
93057,1190416,Flavio Rodrigues De Souza,UTC,2024-03-31T21:00:00+00:00,1711918800,1.711919e+09,1.711922e+09,10494.0,Estádio Urbano Caldeira,"Santos, São Paulo",...,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,True,2025


In [40]:
post_saved_fixtures

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1208021,R. Jones,UTC,2024-08-16T19:00:00+00:00,1723834800,1.723835e+09,1.723838e+09,556.0,Old Trafford,Manchester,...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,None,None
1,1208022,T. Robinson,UTC,2024-08-17T11:30:00+00:00,1723894200,1.723894e+09,1.723898e+09,545.0,Portman Road,"Ipswich, Suffolk",...,0.0,2.0,0.0,0.0,0.0,2.0,NaN,NaN,None,None
2,1208025,C. Pawson,UTC,2024-08-17T14:00:00+00:00,1723903200,1.723903e+09,1.723907e+09,562.0,St. James' Park,Newcastle upon Tyne,...,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,None,None
3,1208023,J. Gillett,UTC,2024-08-17T14:00:00+00:00,1723903200,1.723903e+09,1.723907e+09,494.0,Emirates Stadium,London,...,2.0,0.0,1.0,0.0,2.0,0.0,NaN,NaN,None,None
4,1208024,S. Hooper,UTC,2024-08-17T14:00:00+00:00,1723903200,1.723903e+09,1.723907e+09,8560.0,Goodison Park,Liverpool,...,0.0,3.0,0.0,1.0,0.0,3.0,NaN,NaN,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11120,1354223,Edina Alves Batista,UTC,2025-03-03T23:00:00+00:00,1741042800,1.741043e+09,1.741046e+09,20813.0,MorumBIS,"São Paulo, São Paulo",...,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,None,None
11121,1357839,Matheus Delgado Candançan,UTC,2025-03-09T21:30:00+00:00,1741555800,1.741556e+09,1.741559e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,2.0,1.0,1.0,1.0,2.0,1.0,NaN,NaN,None,None
11122,1358545,Flavio Rodrigues De Souza,UTC,2025-03-11T00:35:00+00:00,1741653300,1.741653e+09,1.741657e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,None,None
11123,1360313,Raphael Claus,UTC,2025-03-16T21:30:00+00:00,1742160600,1.742161e+09,1.742164e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,0.0,1.0,0.0,0.0,0.0,1.0,NaN,NaN,None,None


In [41]:
saved_fixtures_df= pd.concat([prior_saved_fixtures,post_saved_fixtures]).drop("max_season",axis=1)
saved_fixtures_df

C:\Users\Arda\AppData\Local\Temp\ipykernel_4220\1537821446.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  saved_fixtures_df= pd.concat([prior_saved_fixtures,post_saved_fixtures]).drop("max_season",axis=1)


,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings
0,185545,"Fredy Fautrel, France",UTC,2015-08-07T18:30:00+00:00,1438972200,1.438972e+09,1.438976e+09,NaN,Stade Pierre Mauroy,Lille,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,185546,"Wilfried Bien, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Stade de la Beaujoire,Nantes,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,185547,"Hakim Ben el Salem Hadj, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,660.0,Stade de la Mosson,Montpellier,...,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,185548,"Lionel Jaffredo, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Allianz Rivera,Nice,...,1.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,185549,"Frank Schneider, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,667.0,Orange Velodrome,Marseille,...,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11120,1354223,Edina Alves Batista,UTC,2025-03-03T23:00:00+00:00,1741042800,1.741043e+09,1.741046e+09,20813.0,MorumBIS,"São Paulo, São Paulo",...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,7.0,True
11121,1357839,Matheus Delgado Candançan,UTC,2025-03-09T21:30:00+00:00,1741555800,1.741556e+09,1.741559e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,9.0,True
11122,1358545,Flavio Rodrigues De Souza,UTC,2025-03-11T00:35:00+00:00,1741653300,1.741653e+09,1.741657e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,9.0,True
11123,1360313,Raphael Claus,UTC,2025-03-16T21:30:00+00:00,1742160600,1.742161e+09,1.742164e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,7.0,True


In [42]:
#We covert json format to dataframe and save it   

saved_fixtures_df.to_csv("{}saved_fixtures.csv".format(data_dir),index=False)

## Fixture Stats

In [43]:
#In this section we will download given fixture stats.
#There is two limits: Api Calls Per minute(450/min) and Api Calls Quota for a day(75000)

In [44]:
saved_fixtures_df

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings
0,185545,"Fredy Fautrel, France",UTC,2015-08-07T18:30:00+00:00,1438972200,1.438972e+09,1.438976e+09,NaN,Stade Pierre Mauroy,Lille,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,185546,"Wilfried Bien, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Stade de la Beaujoire,Nantes,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,185547,"Hakim Ben el Salem Hadj, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,660.0,Stade de la Mosson,Montpellier,...,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,185548,"Lionel Jaffredo, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Allianz Rivera,Nice,...,1.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,185549,"Frank Schneider, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,667.0,Orange Velodrome,Marseille,...,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11120,1354223,Edina Alves Batista,UTC,2025-03-03T23:00:00+00:00,1741042800,1.741043e+09,1.741046e+09,20813.0,MorumBIS,"São Paulo, São Paulo",...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,7.0,True
11121,1357839,Matheus Delgado Candançan,UTC,2025-03-09T21:30:00+00:00,1741555800,1.741556e+09,1.741559e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,9.0,True
11122,1358545,Flavio Rodrigues De Souza,UTC,2025-03-11T00:35:00+00:00,1741653300,1.741653e+09,1.741657e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,9.0,True
11123,1360313,Raphael Claus,UTC,2025-03-16T21:30:00+00:00,1742160600,1.742161e+09,1.742164e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,7.0,True


In [45]:
saved_fixtures_df.columns

Index(['fixture.id', 'fixture.referee', 'fixture.timezone', 'fixture.date',
       'fixture.timestamp', 'fixture.periods.first', 'fixture.periods.second',
       'fixture.venue.id', 'fixture.venue.name', 'fixture.venue.city',
       'fixture.status.long', 'fixture.status.short', 'fixture.status.elapsed',
       'league.id', 'league.name', 'league.country', 'league.logo',
       'league.season', 'league.round', 'teams.home.id', 'teams.home.name',
       'teams.home.winner', 'teams.away.id', 'teams.away.name',
       'teams.away.winner', 'goals.home', 'goals.away', 'score.halftime.home',
       'score.halftime.away', 'score.fulltime.home', 'score.fulltime.away',
       'score.extratime.home', 'score.extratime.away', 'score.penalty.home',
       'score.penalty.away', 'fixture.status.extra', 'league.standings'],
      dtype='object')

In [46]:
saved_fixtures_df["fixture.status.long"].unique()

array(['Match Finished', 'Match Cancelled', 'Match Postponed',
       'Technical loss', 'Not Started', 'Match Abandoned',
       'Time to be defined'], dtype=object)

In [47]:
finished_matches=['Match Finished', 'Technical loss' ]
finished_matches

['Match Finished', 'Technical loss']

In [48]:
relevant_saved_fixtures=saved_fixtures_df[(saved_fixtures_df["fixture.status.long"].isin(finished_matches))
                                         & (saved_fixtures_df["league.season"]>=2022)]
relevant_saved_fixtures

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings
2664,871470,F. Letexier,UTC,2022-08-06T15:00:00+00:00,1659798000,1.659798e+09,1.659802e+09,681.0,Stade de la Meinau,Strasbourg,...,0.0,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2665,871471,B. Dechepy,UTC,2022-08-07T15:05:00+00:00,1659884700,1.659885e+09,1.659888e+09,680.0,Roazhon Park,Rennes,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2666,871472,J. Stinat,UTC,2022-08-06T19:00:00+00:00,1659812400,1.659812e+09,1.659816e+09,644.0,Stade Gabriel Montpied,Clermont-Ferrand,...,0.0,3.0,0.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
2667,871473,J. Hamel,UTC,2022-08-07T11:00:00+00:00,1659870000,1.659870e+09,1.659874e+09,682.0,Stadium de Toulouse,Toulouse,...,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2668,871474,B. Bastien,UTC,2022-08-05T19:00:00+00:00,1659726000,1.659726e+09,1.659730e+09,666.0,Groupama Stadium,Décines-Charpieu,...,2.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,1354224,Raphael Claus,UTC,2025-03-02T23:45:00+00:00,1740959100,1.740959e+09,1.740963e+09,10494.0,Estádio Urbano Caldeira,"Santos, São Paulo",...,1.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,6.0,True
11120,1354223,Edina Alves Batista,UTC,2025-03-03T23:00:00+00:00,1741042800,1.741043e+09,1.741046e+09,20813.0,MorumBIS,"São Paulo, São Paulo",...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,7.0,True
11121,1357839,Matheus Delgado Candançan,UTC,2025-03-09T21:30:00+00:00,1741555800,1.741556e+09,1.741559e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,9.0,True
11122,1358545,Flavio Rodrigues De Souza,UTC,2025-03-11T00:35:00+00:00,1741653300,1.741653e+09,1.741657e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,9.0,True


In [49]:
call_rate=60/300

In [51]:
quota=api.quota_info()-10

used : 43, quota : 7500, remains : 7457


In [52]:
try:
    
    fixture_stat=pd.read_csv("{}fixture_stat.csv".format(data_dir))
    
except:
    
    fixture_stat=[]


In [53]:
full_stats=fixture_stat
full_stats

,team.id,team.name,team.logo,statistics.Shots on Goal,statistics.Shots off Goal,statistics.Total Shots,statistics.Blocked Shots,statistics.Shots insidebox,statistics.Shots outsidebox,statistics.Fouls,...,statistics.Assists,statistics.Counter Attacks,statistics.Cross Attacks,statistics.Free Kicks,statistics.Goals,statistics.Goal Attempts,statistics.Substitutions,statistics.Throwins,statistics.Medical Treatment,statistics.goals_prevented
0,79,Lille,https://media-2.api-sports.io/football/teams/7...,2.0,4.0,12.0,6.0,4.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85,Paris Saint Germain,https://media-1.api-sports.io/football/teams/8...,3.0,2.0,7.0,2.0,6.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83,Nantes,https://media-1.api-sports.io/football/teams/8...,4.0,2.0,9.0,3.0,4.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90,Guingamp,https://media-2.api-sports.io/football/teams/9...,4.0,4.0,10.0,2.0,5.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82,Montpellier,https://media-2.api-sports.io/football/teams/8...,4.0,7.0,11.0,NaN,9.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164916,2323,Universidad de Chile,https://media.api-sports.io/football/teams/232...,3.0,10.0,18.0,5.0,11.0,7.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164917,131,Corinthians,https://media.api-sports.io/football/teams/131...,4.0,5.0,10.0,1.0,6.0,4.0,16.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164918,128,Santos,https://media.api-sports.io/football/teams/128...,2.0,6.0,14.0,6.0,6.0,8.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164919,121,Palmeiras,https://media.api-sports.io/football/teams/121...,8.0,3.0,14.0,3.0,5.0,9.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
downloaded_stat_fixs=list(full_stats["fixture.id"].unique()) 

In [55]:
downloaded_stat_fixs=list(full_stats["fixture.id"].unique()) 
not_downloaded_stat_fixs=list(relevant_saved_fixtures["fixture.id"][relevant_saved_fixtures["fixture.id"].isin(downloaded_stat_fixs)==False].sort_values().unique())
len(not_downloaded_stat_fixs)

2665

In [56]:
quota=api.quota_info()-10

fixture_json=[]

iterasyon=0
for i in not_downloaded_stat_fixs:
    
    if iterasyon<quota:
        
        
        current=time.time()
        fixture_json=fixture_json+api.fix_stat(i)
        diff=time.time()-current
        
        if diff > call_rate:
            time.sleep(0.2)

        
    print(iterasyon)   
    iterasyon+=1
post_fixture_stat_df= pd.json_normalize(fixture_json)

used : 43, quota : 7500, remains : 7457
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
26

In [57]:
fixture_stat_df=pd.concat([full_stats,post_fixture_stat_df])

fixture_stat_df

,team.id,team.name,team.logo,statistics.Shots on Goal,statistics.Shots off Goal,statistics.Total Shots,statistics.Blocked Shots,statistics.Shots insidebox,statistics.Shots outsidebox,statistics.Fouls,...,statistics.Assists,statistics.Counter Attacks,statistics.Cross Attacks,statistics.Free Kicks,statistics.Goals,statistics.Goal Attempts,statistics.Substitutions,statistics.Throwins,statistics.Medical Treatment,statistics.goals_prevented
0,79,Lille,https://media-2.api-sports.io/football/teams/7...,2.0,4.0,12.0,6.0,4.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85,Paris Saint Germain,https://media-1.api-sports.io/football/teams/8...,3.0,2.0,7.0,2.0,6.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,83,Nantes,https://media-1.api-sports.io/football/teams/8...,4.0,2.0,9.0,3.0,4.0,6.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90,Guingamp,https://media-2.api-sports.io/football/teams/9...,4.0,4.0,10.0,2.0,5.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82,Montpellier,https://media-2.api-sports.io/football/teams/8...,4.0,7.0,11.0,NaN,9.0,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,2320,O'Higgins,https://media.api-sports.io/football/teams/232...,5.0,8.0,21.0,8.0,9.0,12.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1988,2318,Palestino,https://media.api-sports.io/football/teams/231...,7.0,6.0,15.0,2.0,9.0,6.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1989,2337,Nublense,https://media.api-sports.io/football/teams/233...,3.0,5.0,10.0,2.0,5.0,5.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1990,121,Palmeiras,https://media.api-sports.io/football/teams/121...,8.0,8.0,22.0,6.0,14.0,8.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
fixture_stat_df.to_csv("{}fixture_stat.csv".format(data_dir),index=False)

In [59]:
fixture_stat_df.columns

Index(['team.id', 'team.name', 'team.logo', 'statistics.Shots on Goal',
       'statistics.Shots off Goal', 'statistics.Total Shots',
       'statistics.Blocked Shots', 'statistics.Shots insidebox',
       'statistics.Shots outsidebox', 'statistics.Fouls',
       'statistics.Corner Kicks', 'statistics.Offsides',
       'statistics.Ball Possession', 'statistics.Yellow Cards',
       'statistics.Red Cards', 'statistics.Goalkeeper Saves',
       'statistics.Total passes', 'statistics.Passes accurate',
       'statistics.Passes %', 'fixture.id', 'statistics.expected_goals',
       'statistics.Assists', 'statistics.Counter Attacks',
       'statistics.Cross Attacks', 'statistics.Free Kicks', 'statistics.Goals',
       'statistics.Goal Attempts', 'statistics.Substitutions',
       'statistics.Throwins', 'statistics.Medical Treatment',
       'statistics.goals_prevented'],
      dtype='object')

In [60]:
downloaded=list(fixture_stat_df["fixture.id"].unique())
len(downloaded)

83462

In [61]:
check=saved_fixtures_df[(saved_fixtures_df["fixture.id"].isin(downloaded)) & 
                       (saved_fixtures_df["fixture.status.long"]=='Match Finished')]
check

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,...,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away,fixture.status.extra,league.standings
0,185545,"Fredy Fautrel, France",UTC,2015-08-07T18:30:00+00:00,1438972200,1.438972e+09,1.438976e+09,NaN,Stade Pierre Mauroy,Lille,...,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,185546,"Wilfried Bien, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Stade de la Beaujoire,Nantes,...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,185547,"Hakim Ben el Salem Hadj, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,660.0,Stade de la Mosson,Montpellier,...,0.0,1.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,185548,"Lionel Jaffredo, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,NaN,Allianz Rivera,Nice,...,1.0,0.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,185549,"Frank Schneider, France",UTC,2015-08-08T19:00:00+00:00,1439060400,1.439060e+09,1.439064e+09,667.0,Orange Velodrome,Marseille,...,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,1354224,Raphael Claus,UTC,2025-03-02T23:45:00+00:00,1740959100,1.740959e+09,1.740963e+09,10494.0,Estádio Urbano Caldeira,"Santos, São Paulo",...,1.0,0.0,2.0,0.0,NaN,NaN,NaN,NaN,6.0,True
11120,1354223,Edina Alves Batista,UTC,2025-03-03T23:00:00+00:00,1741042800,1.741043e+09,1.741046e+09,20813.0,MorumBIS,"São Paulo, São Paulo",...,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,7.0,True
11121,1357839,Matheus Delgado Candançan,UTC,2025-03-09T21:30:00+00:00,1741555800,1.741556e+09,1.741559e+09,11531.0,Neo Química Arena,"São Paulo, São Paulo",...,1.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,9.0,True
11122,1358545,Flavio Rodrigues De Souza,UTC,2025-03-11T00:35:00+00:00,1741653300,1.741653e+09,1.741657e+09,258.0,Allianz Parque,"São Paulo, São Paulo",...,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,9.0,True


In [62]:
check.columns

Index(['fixture.id', 'fixture.referee', 'fixture.timezone', 'fixture.date',
       'fixture.timestamp', 'fixture.periods.first', 'fixture.periods.second',
       'fixture.venue.id', 'fixture.venue.name', 'fixture.venue.city',
       'fixture.status.long', 'fixture.status.short', 'fixture.status.elapsed',
       'league.id', 'league.name', 'league.country', 'league.logo',
       'league.season', 'league.round', 'teams.home.id', 'teams.home.name',
       'teams.home.winner', 'teams.away.id', 'teams.away.name',
       'teams.away.winner', 'goals.home', 'goals.away', 'score.halftime.home',
       'score.halftime.away', 'score.fulltime.home', 'score.fulltime.away',
       'score.extratime.home', 'score.extratime.away', 'score.penalty.home',
       'score.penalty.away', 'fixture.status.extra', 'league.standings'],
      dtype='object')

In [63]:
stat_groups=check.groupby(["league.season",'league.id', 'league.name','league.country'],as_index=False)["fixture.id"].count()
stat_groups.sort_values("league.season")

,league.season,league.id,league.name,league.country,fixture.id
0,2014,39,Premier League,England,3
1,2014,188,A-League,Australia,4
13,2015,218,Bundesliga,Austria,165
11,2015,188,A-League,Australia,140
10,2015,140,La Liga,Spain,379
...,...,...,...,...,...
262,2024,72,Serie B,Brazil,380
293,2025,265,Primera División,Chile,38
291,2025,98,J1 League,Japan,58
292,2025,169,Super League,China,16


In [64]:
stat_groups[stat_groups["league.season"]==2022]

,league.season,league.id,league.name,league.country,fixture.id
183,2022,39,Premier League,England,380
184,2022,40,Championship,England,557
185,2022,41,League One,England,557
186,2022,42,League Two,England,557
187,2022,61,Ligue 1,France,380
188,2022,62,Ligue 2,France,379
189,2022,71,Serie A,Brazil,380
190,2022,72,Serie B,Brazil,380
191,2022,78,Bundesliga,Germany,308
192,2022,79,2. Bundesliga,Germany,308


In [65]:
all_groups=saved_fixtures_df[(saved_fixtures_df["fixture.status.long"]=='Match Finished')].groupby(["league.season",'league.id', 'league.name','league.country'],as_index=False)["fixture.id"].count()
all_groups

,league.season,league.id,league.name,league.country,fixture.id
0,2014,39,Premier League,England,380
1,2014,40,Championship,England,557
2,2014,62,Ligue 2,France,380
3,2014,79,2. Bundesliga,Germany,306
4,2014,140,La Liga,Spain,380
...,...,...,...,...,...
325,2024,475,Paulista - A1,Brazil,104
326,2025,98,J1 League,Japan,58
327,2025,169,Super League,China,16
328,2025,265,Primera División,Chile,38


In [66]:
stat_groups[stat_groups["league.season"]==2023]

,league.season,league.id,league.name,league.country,fixture.id
219,2023,39,Premier League,England,380
220,2023,40,Championship,England,554
221,2023,41,League One,England,556
222,2023,42,League Two,England,557
223,2023,61,Ligue 1,France,308
224,2023,62,Ligue 2,France,378
225,2023,71,Serie A,Brazil,380
226,2023,72,Serie B,Brazil,380
227,2023,78,Bundesliga,Germany,281
228,2023,79,2. Bundesliga,Germany,307


In [67]:
all_groups[all_groups["league.season"]==2023]

,league.season,league.id,league.name,league.country,fixture.id
254,2023,39,Premier League,England,380
255,2023,40,Championship,England,557
256,2023,41,League One,England,557
257,2023,42,League Two,England,557
258,2023,61,Ligue 1,France,308
259,2023,62,Ligue 2,France,382
260,2023,71,Serie A,Brazil,380
261,2023,72,Serie B,Brazil,380
262,2023,78,Bundesliga,Germany,308
263,2023,79,2. Bundesliga,Germany,308


In [68]:
try:
    
    fixture_event=pd.read_csv("{}fixture_event_df.csv".format(data_dir))
    
except:
    
    fixture_event=[]
   

In [69]:
prior_fixture_event=fixture_event

In [70]:
downloaded_stat_events=list(prior_fixture_event["fixture.id"].unique()) 
not_downloaded_stat_events=list(relevant_saved_fixtures["fixture.id"][relevant_saved_fixtures["fixture.id"].isin(downloaded_stat_events)==False].sort_values().unique())
len(not_downloaded_stat_events)

336

In [71]:

quota=api.quota_info()-10

iterasyon=0
post_event_json=[]

for i in not_downloaded_stat_events:
    
    if iterasyon<quota:
        
        
        current=time.time()
        post_event_json=post_event_json+api.fix_events(i)
        diff=time.time()-current
        
        if diff < call_rate:
            time.sleep(0.2)
        
        

    print(iterasyon,i)   
    iterasyon+=1
post_fixture_event_df= pd.json_normalize(post_event_json).drop("team.logo",axis=1)

used : 840, quota : 7500, remains : 6660
0 828579
1 884549
2 884555
3 884560
4 884563
5 884568
6 884573
7 884577
8 884580
9 884586
10 884591
11 884593
12 884599
13 884609
14 884610
15 884614
16 884623
17 884627
18 884629
19 884634
20 884641
21 884645
22 884653
23 884654
24 884656
25 884659
26 884669
27 884672
28 884675
29 884677
30 927042
31 927044
32 927045
33 927047
34 927053
35 927059
36 1186236
37 1202772
38 1202773
39 1202774
40 1202776
41 1203166
42 1203167
43 1203168
44 1203170
45 1203172
46 1203173
47 1203174
48 1205550
49 1205551
50 1205552
51 1205553
52 1205554
53 1205555
54 1208303
55 1208304
56 1208306
57 1208307
58 1208308
59 1208309
60 1208310
61 1208312
62 1208730
63 1208731
64 1208732
65 1208733
66 1208734
67 1208735
68 1208736
69 1208737
70 1208738
71 1208739
72 1208740
73 1208741
74 1208742
75 1208743
76 1208744
77 1208745
78 1210116
79 1210117
80 1210118
81 1210119
82 1210120
83 1210121
84 1210122
85 1210123
86 1210124
87 1210125
88 1210126
89 1210127
90 1210128
91 1

In [72]:
fixture_event_df=pd.concat([prior_fixture_event,post_fixture_event_df])

In [73]:

fixture_event_df.to_csv("{}fixture_event_df.csv".format(data_dir),index=False)